# Benchmark: Chunk I/O & File Format Save Performance

Tests:
1. Read performance at different chunk sizes (1 - 1000 MB)
2. Save performance to different file formats using mbo_utilities.imwrite
3. Scan phase estimation with FFT method

**Reference Dataset:** `\\rbo-s1\S1_DATA\lbm\kbarber\2025-11-04-mk311\raw\green`

In [ ]:
from datetime import date
import mbo_utilities
import platform
import os
import tempfile
import shutil

BENCHMARK_DATE = date.today().isoformat()

print(f"Benchmark Date: {BENCHMARK_DATE}")
print(f"mbo_utilities: {mbo_utilities.__version__}")
print()

print("=" * 60)
print("SYSTEM INFO")
print("=" * 60)
print(f"OS: {platform.system()} {platform.release()}")
print(f"Python: {platform.python_version()}")
print(f"CPU: {platform.processor()}")
print(f"CPU cores: {os.cpu_count()}")

try:
    import psutil
    mem = psutil.virtual_memory()
    print(f"RAM: {mem.total / 1024**3:.1f} GB total, {mem.available / 1024**3:.1f} GB available ({mem.percent}% used)")
    disk = psutil.disk_usage('C:/')
    print(f"Disk (C:): {disk.total / 1024**3:.0f} GB total, {disk.free / 1024**3:.0f} GB free")
except ImportError:
    print("(install psutil for memory/disk info)")

import numpy as np
print(f"NumPy: {np.__version__}")
print("=" * 60)

In [ ]:
from pathlib import Path
import time
import numpy as np
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

from mbo_utilities.arrays.tiff import MboRawArray
from mbo_utilities.phasecorr import _phase_corr_2d
from mbo_utilities import imwrite

# reference dataset
TEST_PATH = r"\\rbo-s1\S1_DATA\lbm\kbarber\2025-11-04-mk311\raw\green"

# test parameters
N_FRAMES = 2000
Z_PLANE = 0

In [ ]:
p = Path(TEST_PATH)
files = sorted(list(p.glob("*.tif")) + list(p.glob("*.tiff")))
print(f"Found {len(files)} files")

arr = MboRawArray(files=files)
arr.fix_phase = False
print(f"Shape: {arr.shape}")
print(f"Dtype: {arr.dtype}")
print(f"Z-planes: {arr.num_channels}")

# frame size
frame_bytes = arr.shape[-2] * arr.shape[-1] * np.dtype(arr.dtype).itemsize
frame_mb = frame_bytes / 1024 / 1024
print(f"\nFrame size: {frame_bytes:,} bytes ({frame_mb:.3f} MB)")
print(f"Test frames: {N_FRAMES} ({N_FRAMES * frame_mb:.1f} MB total)")

## 1. Chunk Size Read Benchmark

Test read throughput at different chunk sizes (MB).

In [ ]:
# chunk sizes in MB
target_mb = [1, 10, 50, 100, 250, 500, 1000]

# convert to frames, cap at N_FRAMES
chunk_configs = []
for mb in target_mb:
    n_frames = max(1, int(mb / frame_mb))
    if n_frames <= N_FRAMES:
        actual_mb = n_frames * frame_mb
        chunk_configs.append({'target_mb': mb, 'frames': n_frames, 'actual_mb': actual_mb})

print(f"{'Target MB':<12} {'Frames':<10} {'Actual MB':<12}")
print("-" * 35)
for c in chunk_configs:
    print(f"{c['target_mb']:<12} {c['frames']:<10} {c['actual_mb']:<12.2f}")

In [ ]:
# benchmark reads
read_results = []
n_repeats = 3

for cfg in tqdm(chunk_configs, desc="Chunk sizes"):
    n = cfg['frames']
    times = []
    
    for rep in range(n_repeats):
        # random start to avoid cache
        max_start = max(0, min(arr.num_frames, N_FRAMES) - n - 10)
        start = np.random.randint(0, max(1, max_start))
        
        t0 = time.perf_counter()
        data = arr[start:start+n, Z_PLANE]
        t1 = time.perf_counter()
        times.append(t1 - t0)
    
    avg_time = np.mean(times)
    read_results.append({
        'target_mb': cfg['target_mb'],
        'frames': n,
        'actual_mb': cfg['actual_mb'],
        'time_sec': avg_time,
        'throughput_mb_s': cfg['actual_mb'] / avg_time,
        'ms_per_frame': avg_time * 1000 / n
    })

print("\nRead Results:")
print(f"{'MB':<8} {'Frames':<8} {'Time (s)':<10} {'MB/s':<10} {'ms/frame':<10}")
print("-" * 50)
for r in read_results:
    print(f"{r['target_mb']:<8} {r['frames']:<8} {r['time_sec']:<10.2f} {r['throughput_mb_s']:<10.1f} {r['ms_per_frame']:<10.2f}")

In [ ]:
# plot read benchmark
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

mb_vals = [r['target_mb'] for r in read_results]
throughput = [r['throughput_mb_s'] for r in read_results]
ms_per_frame = [r['ms_per_frame'] for r in read_results]

# throughput vs chunk size
ax = axes[0]
bars = ax.bar(range(len(mb_vals)), throughput, color='steelblue', alpha=0.8)
ax.set_xticks(range(len(mb_vals)))
ax.set_xticklabels([f"{m}" for m in mb_vals])
ax.set_xlabel('Chunk Size (MB)')
ax.set_ylabel('Throughput (MB/s)')
ax.set_title('Read Throughput vs Chunk Size')
ax.grid(True, alpha=0.3, axis='y')

for bar, val in zip(bars, throughput):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1, 
            f'{val:.0f}', ha='center', va='bottom', fontsize=9)

# ms per frame vs chunk size
ax = axes[1]
bars = ax.bar(range(len(mb_vals)), ms_per_frame, color='coral', alpha=0.8)
ax.set_xticks(range(len(mb_vals)))
ax.set_xticklabels([f"{m}" for m in mb_vals])
ax.set_xlabel('Chunk Size (MB)')
ax.set_ylabel('Time per Frame (ms)')
ax.set_title('Per-Frame Read Time vs Chunk Size')
ax.grid(True, alpha=0.3, axis='y')

for bar, val in zip(bars, ms_per_frame):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.05, 
            f'{val:.2f}', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

## 2. File Format Save Benchmark (mbo_utilities.imwrite)

Test save performance using mbo_utilities.imwrite to different formats.

In [ ]:
# temp directory
temp_dir = Path(tempfile.mkdtemp(prefix="mbo_benchmark_"))
print(f"Temp dir: {temp_dir}")
print(f"Testing with {N_FRAMES} frames, plane {Z_PLANE}")

save_results = []

In [ ]:
# TIFF format
tiff_dir = temp_dir / "tiff_out"
tiff_dir.mkdir()

t0 = time.perf_counter()
imwrite(arr, tiff_dir, ext=".tiff", planes=[Z_PLANE + 1], num_frames=N_FRAMES, overwrite=True)
t1 = time.perf_counter()

tiff_files = list(tiff_dir.glob("*.tiff")) + list(tiff_dir.glob("*.tif"))
tiff_size = sum(f.stat().st_size for f in tiff_files) / 1024 / 1024

save_results.append({
    'format': 'TIFF (.tiff)',
    'time_sec': t1 - t0,
    'size_mb': tiff_size,
    'throughput_mb_s': tiff_size / (t1 - t0)
})
print(f"TIFF: {t1-t0:.2f}s, {tiff_size:.1f} MB, {tiff_size/(t1-t0):.1f} MB/s")

In [ ]:
# Zarr format
zarr_dir = temp_dir / "zarr_out"
zarr_dir.mkdir()

t0 = time.perf_counter()
imwrite(arr, zarr_dir, ext=".zarr", planes=[Z_PLANE + 1], num_frames=N_FRAMES, overwrite=True)
t1 = time.perf_counter()

zarr_stores = list(zarr_dir.glob("*.zarr"))
zarr_size = sum(sum(f.stat().st_size for f in store.rglob('*') if f.is_file()) for store in zarr_stores) / 1024 / 1024

save_results.append({
    'format': 'Zarr (.zarr)',
    'time_sec': t1 - t0,
    'size_mb': zarr_size,
    'throughput_mb_s': zarr_size / (t1 - t0)
})
print(f"Zarr: {t1-t0:.2f}s, {zarr_size:.1f} MB, {zarr_size/(t1-t0):.1f} MB/s")

In [ ]:
# HDF5 format
h5_dir = temp_dir / "h5_out"
h5_dir.mkdir()

t0 = time.perf_counter()
imwrite(arr, h5_dir, ext=".h5", planes=[Z_PLANE + 1], num_frames=N_FRAMES, overwrite=True)
t1 = time.perf_counter()

h5_files = list(h5_dir.glob("*.h5")) + list(h5_dir.glob("*.hdf5"))
h5_size = sum(f.stat().st_size for f in h5_files) / 1024 / 1024

save_results.append({
    'format': 'HDF5 (.h5)',
    'time_sec': t1 - t0,
    'size_mb': h5_size,
    'throughput_mb_s': h5_size / (t1 - t0)
})
print(f"HDF5: {t1-t0:.2f}s, {h5_size:.1f} MB, {h5_size/(t1-t0):.1f} MB/s")

In [ ]:
# Suite2p binary format
bin_dir = temp_dir / "bin_out"
bin_dir.mkdir()

t0 = time.perf_counter()
imwrite(arr, bin_dir, ext=".bin", planes=[Z_PLANE + 1], num_frames=N_FRAMES, overwrite=True)
t1 = time.perf_counter()

bin_files = list(bin_dir.rglob("*.bin"))
bin_size = sum(f.stat().st_size for f in bin_files) / 1024 / 1024

save_results.append({
    'format': 'Binary (.bin)',
    'time_sec': t1 - t0,
    'size_mb': bin_size,
    'throughput_mb_s': bin_size / (t1 - t0)
})
print(f"Binary: {t1-t0:.2f}s, {bin_size:.1f} MB, {bin_size/(t1-t0):.1f} MB/s")

In [ ]:
# plot save results
fig, axes = plt.subplots(1, 3, figsize=(14, 5))

formats = [r['format'] for r in save_results]
times = [r['time_sec'] for r in save_results]
sizes = [r['size_mb'] for r in save_results]
throughputs = [r['throughput_mb_s'] for r in save_results]

colors = plt.cm.Set2(np.linspace(0, 1, len(formats)))

# save time
ax = axes[0]
bars = ax.barh(range(len(formats)), times, color=colors)
ax.set_yticks(range(len(formats)))
ax.set_yticklabels(formats)
ax.set_xlabel('Save Time (seconds)')
ax.set_title('Save Time by Format')
ax.grid(True, alpha=0.3, axis='x')
for bar, val in zip(bars, times):
    ax.text(bar.get_width() + 0.1, bar.get_y() + bar.get_height()/2, 
            f'{val:.1f}s', va='center', fontsize=10)

# file size
ax = axes[1]
bars = ax.barh(range(len(formats)), sizes, color=colors)
ax.set_yticks(range(len(formats)))
ax.set_yticklabels(formats)
ax.set_xlabel('File Size (MB)')
ax.set_title('File Size by Format')
ax.grid(True, alpha=0.3, axis='x')
for bar, val in zip(bars, sizes):
    ax.text(bar.get_width() + 1, bar.get_y() + bar.get_height()/2, 
            f'{val:.0f}', va='center', fontsize=10)

# throughput
ax = axes[2]
bars = ax.barh(range(len(formats)), throughputs, color=colors)
ax.set_yticks(range(len(formats)))
ax.set_yticklabels(formats)
ax.set_xlabel('Throughput (MB/s)')
ax.set_title('Write Throughput by Format')
ax.grid(True, alpha=0.3, axis='x')
for bar, val in zip(bars, throughputs):
    ax.text(bar.get_width() + 1, bar.get_y() + bar.get_height()/2, 
            f'{val:.0f}', va='center', fontsize=10)

plt.tight_layout()
plt.show()

## 3. Scan Phase Estimation (FFT Method)

In [ ]:
# load test data for phase analysis
print(f"Loading {N_FRAMES} frames for phase analysis...")
t0 = time.perf_counter()
test_data = arr[:N_FRAMES, Z_PLANE]
t1 = time.perf_counter()
print(f"Loaded in {t1-t0:.1f}s, shape: {test_data.shape}")

In [ ]:
# test scan phase with different mean windows using FFT
window_sizes = [1, 10, 100, 1000]
num_samples = 5

phase_results = []

for ws in tqdm(window_sizes, desc="Window sizes"):
    if ws > N_FRAMES:
        continue
        
    offsets = []
    times = []
    
    n_possible = N_FRAMES // ws
    n_samp = min(num_samples, n_possible)
    starts = np.linspace(0, N_FRAMES - ws, n_samp, dtype=int)
    
    for start in starts:
        t0 = time.perf_counter()
        
        frames = test_data[start:start+ws]
        mean_frame = np.mean(frames, axis=0)
        
        # FFT phase estimation
        offset = _phase_corr_2d(mean_frame, upsample=10, border=4, max_offset=10, use_fft=True)
        
        t1 = time.perf_counter()
        offsets.append(offset)
        times.append(t1 - t0)
    
    phase_results.append({
        'window': ws,
        'mean_offset': np.mean(offsets),
        'std_offset': np.std(offsets),
        'time_ms': np.mean(times) * 1000
    })
    
    print(f"Window {ws:5d}: offset = {np.mean(offsets):+.3f} +/- {np.std(offsets):.3f} px, time = {np.mean(times)*1000:.1f} ms")

In [ ]:
# plot phase results
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

windows = [r['window'] for r in phase_results]
offsets = [r['mean_offset'] for r in phase_results]
stds = [r['std_offset'] for r in phase_results]
times_ms = [r['time_ms'] for r in phase_results]

# offset with error bars
ax = axes[0]
ax.errorbar(windows, offsets, yerr=stds, fmt='o-', capsize=8, capthick=2, 
            markersize=10, linewidth=2, color='steelblue')
ax.set_xscale('log')
ax.set_xlabel('Window Size (frames)', fontsize=12)
ax.set_ylabel('Phase Offset (pixels)', fontsize=12)
ax.set_title('Phase Offset vs Mean Window Size (FFT method)', fontsize=12)
ax.grid(True, alpha=0.3)
ax.set_xticks(windows)
ax.set_xticklabels([str(w) for w in windows])

for w, o, s in zip(windows, offsets, stds):
    ax.annotate(f'{o:.2f}+/-{s:.2f}', (w, o), textcoords='offset points', 
                xytext=(0, 15), ha='center', fontsize=9)

# computation time
ax = axes[1]
bars = ax.bar(range(len(windows)), times_ms, color='coral', alpha=0.8)
ax.set_xticks(range(len(windows)))
ax.set_xticklabels([str(w) for w in windows])
ax.set_xlabel('Window Size (frames)', fontsize=12)
ax.set_ylabel('Computation Time (ms)', fontsize=12)
ax.set_title('Phase Computation Time (data in memory)', fontsize=12)
ax.grid(True, alpha=0.3, axis='y')

for bar, val in zip(bars, times_ms):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5, 
            f'{val:.1f}', ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.show()

## Summary

In [ ]:
# expected raw size
raw_size_mb = N_FRAMES * frame_mb

print("=" * 70)
print("CHUNK & SCANPHASE BENCHMARK SUMMARY")
print("=" * 70)
print(f"Date: {BENCHMARK_DATE}")
print(f"mbo_utilities: {mbo_utilities.__version__}")
print(f"Test data: {N_FRAMES} frames, {raw_size_mb:.1f} MB")
print()

# best read chunk
best_read = max(read_results, key=lambda x: x['throughput_mb_s'])
print(f"Best read chunk: {best_read['target_mb']} MB ({best_read['throughput_mb_s']:.0f} MB/s)")
print()

# save performance
print("Save performance (mbo_utilities.imwrite):")
for r in sorted(save_results, key=lambda x: x['time_sec']):
    print(f"  {r['format']:<18} {r['time_sec']:>6.1f}s  {r['size_mb']:>7.0f} MB  {r['throughput_mb_s']:>6.0f} MB/s")
print()

# phase offset
if phase_results:
    best = phase_results[-1]
    print(f"Phase offset ({best['window']}-frame mean): {best['mean_offset']:.3f} +/- {best['std_offset']:.3f} px")
print("=" * 70)

In [ ]:
# cleanup
shutil.rmtree(temp_dir, ignore_errors=True)
print(f"Cleaned up {temp_dir}")

for tf in arr.tiff_files:
    tf.close()